In [1]:
# Importing all the Necessary Libraries and Modules
import numpy as np
import pandas as pd
import scipy.io
from sklearn import svm, pipeline, base, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel
import functools
import os.path, zipfile
import matplotlib.pyplot as plt
import warnings
from scipy import signal
import ImportantModulesForPSO as imp

In [2]:
# Loading Dataset

# Read DataSet BCI Competition III, DataSet IVa - Training
sub1_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub1\100Hz\data_set_IVa_aa.mat", struct_as_record=True)
sub2_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub2\100Hz\data_set_IVa_al.mat", struct_as_record=True)
sub3_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub3\100Hz\data_set_IVa_av.mat", struct_as_record=True)
sub4_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub4\100Hz\data_set_IVa_aw.mat", struct_as_record=True)
sub5_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub5\100Hz\data_set_IVa_ay.mat", struct_as_record=True)


# Read DataSet BCI Competition III, DataSet IVa - True Label
sub1_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_aa.mat", struct_as_record=True)
sub2_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_al.mat", struct_as_record=True)
sub3_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_av.mat", struct_as_record=True)
sub4_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_aw.mat", struct_as_record=True)
sub5_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_ay.mat", struct_as_record=True)


In [204]:
# Extracting Important Information as Global Variable

# SciPy.io.loadmat does not deal well with Matlab structures, resulting in lots of
# extra dimensions in the arrays. This makes the code a bit more cluttered


m = sub5_100hz_training



sample_rate = m['nfo']['fs'][0][0][0][0]
#EEG = m['cnt'].T
#nchannels_yt, nsamples_yt = EEG.shape

#channel_names = [s[0] for s in m['nfo']['clab'][0][0][0]]
event_onsets = m['mrk'][0][0][0]
event_codes = m['mrk'][0][0][1]
#labels = np.zeros((1, nsamples), int)
#labels[0, event_onsets] = event_codes

cl_lab = [s[0] for s in m['mrk']['className'][0][0][0]]
cl1 = cl_lab[0]
cl2 = cl_lab[1]
nclasses = len(cl_lab)
nevents = len(event_onsets)

In [205]:
# Extract Data
d = imp.load_mat(sub5_100hz_training,sub5_100hz_true_label,None)

In [206]:
# Extract Trials
trials = imp.ExtractTrial(d['X'],event_onsets,sample_rate,cl_lab,event_codes)

(118, 300, 18) (118, 300, 10)


In [207]:
# Apply BPF to Data
trials_filt = {
    cl1 : imp.bandpass(trials[cl1],5,20,sample_rate),
    cl2 : imp.bandpass(trials[cl2],5,20,sample_rate)
}

In [208]:
# Initialize Controlling Parameters for PSO  ( HyperParameter)
N = 30
c1 = 2
c2 = 2
Wmin = 0.4
Wmax = 0.9
Vmax = 5
MaxIter = 50

In [215]:
# Initialize Parameters

options = {'N': 15,'c1': 2, 'c2': 2, 'Wmax':0.9,'Wmin': 0.4,'Vmax': 5,'MaxIter' :10,'Threshold': 0.50,'test_size':0.33}
print(len(d['X']))

118


In [216]:
# Initialize Population and Velocity Randomly
Population = dict()
Velocity = dict()
for i in range(0,options['N']):
    Population[i] = np.random.randint(0,2,size = len(d['X']))
    Velocity[i] = np.random.randn(len(d['X']))
#print(Population,Velocity)




In [221]:
mu = 0.8
k = 15

In [226]:
Total_PBS_Score = list(list())
Total_GBS_Score = list()

In [222]:
# Apply PSO in the Following Way -
t = 0
n = 0
PBS_Accuracy = list()
PBL_Accuracy = list()
GBS = 0
GBL = list()
# Create a Do While Loop 
while(t<options['MaxIter']):
    
    # Breaking Condition :
    if(t == options['MaxIter'] ): # or (1- GBS)< 0.015 
        print("Loop should Break Here") 
        break
    
    
    print(str(t+1) + "th iteration")
    Accuracy = list()
    # First Loop [ For Each Particle ]
    for i in range(0,options['N']):
        temp1 = []
        temp2 = []
        for j in range(0,len(Population[i])):
            if(Population[i][j] == 1):
                temp1.append(trials_filt[cl1][j]),
                temp2.append(trials_filt[cl2][j])
        len1 = len(temp1)
        temp1 = np.asarray(temp1)
        temp2 = np.asarray(temp2)
        pso_filt = dict()
        #print("temp1.shape,temp2.shape")
        #print(temp1.shape,temp2.shape)
        pso_filt = {
            cl1 : temp1,
            cl2 : temp2
        }
        # Apply SCSP to Each Particle
        SCSP = dict()
        SCSP_Combined = dict()
        SCSPVar = dict()
        X_SCSP = dict()
        W = dict()
        #print(i)
        W = imp.scsp(pso_filt[cl1],pso_filt[cl2],mu,k)
        SCSP = {cl1 : imp.apply_mix(W,pso_filt[cl1],np.array(pso_filt[cl1])),
                cl2 : imp.apply_mix(W,pso_filt[cl2],np.array(pso_filt[cl2]))
        }
            # Apply SVM and Calc. it's accuracy
        SCSP_Combined = imp.CombineData(SCSP[cl1],SCSP[cl2]) # This is the Combined Data 
        y = imp.CreateY(pso_filt)
        SCSPVar = imp.ChanVar.transform(SCSP_Combined)
        X_SCSP = SCSPVar.T
        #print(X_SCSP[i].shape)
        model,X_test,y_test = imp.SVMModel(X_SCSP,y,options['test_size'])
        Accuracy.append(imp.SVMAccuracy(model,X_test,y_test))
            # Update PBS,GBL if Required
        if( t == 0 ):
            PBS_Accuracy.append(Accuracy[i])
            Total_PBS_Score.append(Accuracy[i])
            PBS = Population[i]
            GBS = max(Accuracy)
        else :
            if(Accuracy[i]>PBS_Accuracy[i]):
                PBS_Accuracy[i] = Accuracy [i]
                Total_PBS_Score.append(Accuracy[i])
                PBS = Population[i]
            if(max(PBS_Accuracy) > GBS):
                GBS = max(PBS_Accuracy)
                GBL = Population[PBS_Accuracy.index(max(PBS_Accuracy))]
                
        print("Best Personal Solution is " + str(PBS_Accuracy[i]) + " For " + str(i) + "th Particle")
        print("Global Best Solution is " + str(GBS))
        
        
    V_next = Velocity    
    # Second Loop [ For Each Particle ]
    for i in range(0,options['N']):
        #print(i)
        # Update intertia
        w = options['Wmax']-((options['Wmax']-options['Wmin'])*t)/options['MaxIter']
        # Update Velocty using the Formula for V[t+1]
        V_next[i] = w*Velocity[i] + options['c1']*np.random.randn()*(Population[PBS_Accuracy.index(max(PBS_Accuracy))]-Population[i]) + options['c2']*np.random.randn()*( Population[PBS_Accuracy.index(GBS)] - Population[i] )
        Velocity[i] = V_next[i]
        # Update Particle Position by applying Sigmoid Function's {0,1} Logic on V[t+1] and changing X[t+1] on that basis
        for j in range(0,len(Population[i])):
            if(imp.sigmoid(V_next[i][j])>=options['Threshold']):
                Population[i][j] = 1
            elif(imp.sigmoid(V_next[i][j])<options['Threshold']):
                Population[i][j] = 0
    t +=1

# Do So until MaxIter or Min Error Criteria is Reached
        
        

1th iteration
18
10
Best Personal Solution is 0.9 For 0th Particle
Global Best Solution is 0.9
18
10
Best Personal Solution is 1.0 For 1th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 2th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 3th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 0.8 For 4th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 0.9 For 5th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 6th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 0.9 For 7th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 0.9 For 8th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 9th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 10th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 11th Particle
Global Best Solution is 1.0
18
10
Best P

18
10
Best Personal Solution is 1.0 For 14th Particle
Global Best Solution is 1.0
8th iteration
18
10
Best Personal Solution is 1.0 For 0th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 1th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 2th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 3th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 4th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 5th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 6th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 7th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 8th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 9th Particle
Global Best Solution is 1.0
18
10
Best Personal Solution is 1.0 For 10th Particle
Global Best Solution is 1.0
18
10
Best P

In [223]:
print(GBS, Population[PBS_Accuracy.index(GBS)])

1.0 [0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0
 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0
 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 1
 1 1 1 1 0 0 0]


In [224]:
counter = 0
for i in range(0,len(Population[PBS_Accuracy.index(GBS)])): 
    if(Population[PBS_Accuracy.index(GBS)][i] == 1):
        print(d['chan_lab'][i])
        counter +=1
print(counter)

Fpz
AF3
AF8
FAF5
F7
F5
F3
F1
F4
F6
F8
FFC7
FFC5
FFC3
FFC1
FFC2
FT7
FC5
FC1
FCz
FC4
FC6
CFC3
C3
Cz
C2
C4
C6
T8
CCP3
CCP6
TP9
CP5
CP1
CPz
CP6
TP10
PCP7
PCP5
PCP3
PCP6
P7
P5
P1
Pz
P2
P4
PPO1
PPO2
PPO8
PO7
PO8
OPO1
OPO2
O1
Oz
O2
OI1
58


### mu = 0 SCSP SUB1
AF3
FAF5
FAF1
Fz
F4
FFC3
FT9
FC6
CFC7
CFC5
CFC1
C3
Cz
CCP3
CCP2
CCP8
CP5
CP1
CP2
TP8
TP10
PCP5
PCP3
PCP1
PCP4
P9
P5
P3
P4
P8
P10
PPO1
PPO8
PO4
PO8
OPO1
OPO2
Oz
OI2
I1
40

### mu = 0 SCSP SUB2
Fpz
AFp2
FAF5
FAF1
FAF2
F3
FFC2
FC3
FC1
FC4
FT8
CFC7
CFC5
CFC1
CFC2
T7
C5
Cz
C4
C6
T8
CCP1
CCP2
CCP4
CCP8
TP7
CP3
CP1
CPz
TP8
PCP5
PCP4
P9
P5
P1
P4
P8
PPO5
PPO1
PPO8
PO7
POz
PO2
O1
I1
I2
46

### mu = 0 SCSP SUB3
Fp2
AF7
AF4
AF8
F3
F8
FFC3
FFC1
FFC2
FFC4
FFC6
FT9
FC5
FCz
FC2
FC4
FC6
FT8
CFC7
CFC5
CFC3
CFC1
T7
C5
Cz
C2
C4
T8
CCP7
CCP5
CCP1
CCP4
CCP6
TP7
CP3
CPz
CP2
CP6
TP8
TP10
PCP7
PCP5
PCP2
PCP4
P3
P1
Pz
P2
P4
P6
P8
PPO5
PPO1
PPO2
PO7
PO3
OPO1
OPO2
Oz
OI2
I1
I2
62

### mu = 0 SCSP SUB4

AFp2
AF3
AF8
FAF5
FAF6
Fz
F8
FFC5
FFC2
FFC4
FT7
FC5
FC1
FC2
FT8
CFC1
CFC4
CFC6
CFC8
T7
C5
C2
CCP7
CCP5
CCP3
CCP1
CCP4
CP3
CPz
TP10
PCP1
PCP6
PCP8
P3
P1
P4
P8
PPO1
PO7
POz
O2
I1
42

### mu = 0 SCSP SUB5

Fpz
AF3
AF8
FAF5
F7
F5
F3
F1
F4
F6
F8
FFC7
FFC5
FFC3
FFC1
FFC2
FT7
FC5
FC1
FCz
FC4
FC6
CFC3
C3
Cz
C2
C4
C6
T8
CCP3
CCP6
TP9
CP5
CP1
CPz
CP6
TP10
PCP7
PCP5
PCP3
PCP6
P7
P5
P1
Pz
P2
P4
PPO1
PPO2
PPO8
PO7
PO8
OPO1
OPO2
O1
Oz
O2
OI1
58

### mu = 0.8 SCSP SUB1
Fpz
AF7
AF3
FAF5
FAF1
F2
F4
FFC7
FT9
FT10
CFC7
CFC5
CFC3
CFC1
C6
CCP2
CCP4
CCP8
CP5
CP1
CPz
CP2
TP8
PCP3
PCP4
P9
P4
P6
PPO1
PPO8
PO3
PO4
PO8
OPO1
OPO2
O1
Oz
OI2
I1
39

### mu = 0.8 SCSP SUB2
F5
F4
CFC8
C3
C1
C4
CCP7
CP5
CP6
PPO6
OPO2
11


Fpz
AFp2
AF3
AF8
FAF2
FAF6
F3
F1
Fz
F6
F8
FFC3
FFC2
FT7
FC5
FC3
FC2
FT8
FT10
CFC3
CFC8
T7
C1
Cz
C2
C6
CCP7
CCP5
CCP2
CCP4
TP9
TP7
CPz
CP4
CP6
TP8
TP10
PCP7
PCP5
PCP3
PCP1
PCP6
PCP8
P9
P7
P5
P1
P2
P6
P8
P10
PPO5
PPO1
PO1
POz
PO8
O1
Oz
O2
OI2
I2
61

### mu = 0.8 SCSP SUB3
Fp1
Fpz
AF3
FAF1
F7
F5
F3
F1
Fz
F6
F8
FFC7
FFC2
FFC6
FFC8
FC5
FC3
FC1
FCz
FC4
FT8
CFC2
CFC4
CFC6
T7
C3
C1
Cz
C4
CCP7
CCP1
CCP2
CCP6
CCP8
TP9
CP3
CP1
CP4
TP8
PCP5
PCP1
PCP2
P9
P5
P1
P2
PPO1
PPO2
POz
PO4
PO8
OPO2
OI1
I1
54


AFp2
Fp2
AF3
AF4
AF8
FAF5
F5
F1
Fz
F2
FFC2
FFC8
FT9
FC1
FC4
FT10
CFC3
CFC2
CFC8
T7
Cz
C4
C6
CCP7
CCP4
CCP6
TP7
CP3
CPz
CP2
CP4
TP10
PCP2
PCP4
PCP6
P9
P5
P3
P2
P4
P6
P10
PPO7
PPO6
PPO8
PO1
POz
PO2
PO8
OPO2
O2
OI2
I1
53

### mu = 0.8 SCSP SUB4

Fp1
AFp1
AF8
FAF6
F5
F1
Fz
F2
FFC5
FFC3
FFC2
FFC6
FT9
FT7
FC5
FC4
FC6
CFC5
CFC6
C3
C1
Cz
C4
T8
CCP3
CCP1
CCP8
TP7
CP6
TP10
PCP6
P9
P3
P2
P8
PPO2
PO7
PO3
O1
Oz
OI2
41

### mu = 0.8 SCSP SUB5

Fpz
AF3
AF8
FAF5
F7
F5
F3
F1
F4
F6
F8
FFC7
FFC5
FFC3
FFC1
FFC2
FT7
FC5
FC1
FCz
FC4
FC6
CFC3
C3
Cz
C2
C4
C6
T8
CCP3
CCP6
TP9
CP5
CP1
CPz
CP6
TP10
PCP7
PCP5
PCP3
PCP6
P7
P5
P1
Pz
P2
P4
PPO1
PPO2
PPO8
PO7
PO8
OPO1
OPO2
O1
Oz
O2
OI1
58

In [14]:
PBL_Accuracy.index(GBS)

ValueError: 1.0 is not in list

In [ ]:
PBL_Accuracy.index(max(PBL_Accuracy))
max(PBL_Accuracy)

In [ ]:
len(PBL_Accuracy)

In [ ]:
for i in range(0,options['N']):
    unique, counts = np.unique(Population[i], return_counts=True)
    print(i,unique,counts)
    
print(trials_filt[cl1].shape,trials_filt[cl2].shape)
